In [30]:
import os
import pandas as pd
import pyreadstat

In [31]:
ruta_carpeta = "data"
ruta_salida = "base_final_unificada_normalizada.sav"

In [32]:
archivos_sav = [f for f in os.listdir(ruta_carpeta) if f.endswith(".sav")]

print(f"Archivos encontrados: {len(archivos_sav)}")
archivos_sav

Archivos encontrados: 11


['201712041525490jPevoWKMXOZPJq5KpYU9Tlj4peGmEEd.sav',
 '20181226143330lNO5qooyMLWxGu8XJLL7BXprdvTjfO0q.sav',
 '20191129151703IbgVgHc5enyxiXopUTuxcrUzC5WPkzCr.sav',
 '20201201154915Yd6FWtyVWNULqHYzwTiXcRaLAl8B8jul.sav',
 '20210730184315Yd6FWtyVWNULqHYzwTiXcRaLAl8B8jul (2).sav',
 '20220729171254FYBQhYtMxyPU3C9wbQbaxGTEtg107w32.sav',
 '202307282030176jUXYrCteX8JWZUbSn6B6fCXUXaZQ78F.sav',
 '2W4N7bnfbaVntLsINTDXCUZ57oCLyC3o.sav',
 'PZrZNHRSBszrytG8VuGha9WbabSmldOl.sav',
 'RceAnfmGqPRS02bqMI8XMzt6ZXx64XS7.sav',
 'RESsr1X3WuQcTxhLVlyslBKGtHAXqSrH.sav']

In [33]:
lista_bases = []
resumen = []

for archivo in archivos_sav:
    try:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        df = pd.read_spss(ruta_archivo)

        columnas_originales = set(df.columns)

        # Normalizar nombres equivalentes
        if "PPERHOM" in df.columns:
            df = df.rename(columns={"PPERHOM": "PUEHOM"})
        if "PPERMUJ" in df.columns:
            df = df.rename(columns={"PPERMUJ": "PUEMUJ"})

        columnas_finales = set(df.columns)

        resumen.append({
            "archivo": archivo,
            "columnas_antes": len(columnas_originales),
            "columnas_despues": len(columnas_finales),
            "renombro_PPER": columnas_originales != columnas_finales
        })

        lista_bases.append(df)

        print(f"Procesado: {archivo}")

    except Exception as e:
        print(f"ERROR con {archivo}: {e}")


Procesado: 201712041525490jPevoWKMXOZPJq5KpYU9Tlj4peGmEEd.sav
Procesado: 20181226143330lNO5qooyMLWxGu8XJLL7BXprdvTjfO0q.sav
Procesado: 20191129151703IbgVgHc5enyxiXopUTuxcrUzC5WPkzCr.sav
Procesado: 20201201154915Yd6FWtyVWNULqHYzwTiXcRaLAl8B8jul.sav
Procesado: 20210730184315Yd6FWtyVWNULqHYzwTiXcRaLAl8B8jul (2).sav
Procesado: 20220729171254FYBQhYtMxyPU3C9wbQbaxGTEtg107w32.sav
Procesado: 202307282030176jUXYrCteX8JWZUbSn6B6fCXUXaZQ78F.sav
Procesado: 2W4N7bnfbaVntLsINTDXCUZ57oCLyC3o.sav
Procesado: PZrZNHRSBszrytG8VuGha9WbabSmldOl.sav
Procesado: RceAnfmGqPRS02bqMI8XMzt6ZXx64XS7.sav
Procesado: RESsr1X3WuQcTxhLVlyslBKGtHAXqSrH.sav


In [34]:
base_final = pd.concat(lista_bases, ignore_index=True)

print(f"Base final creada con {base_final.shape[0]} filas y {base_final.shape[1]} columnas")

Base final creada con 71576 filas y 23 columnas


In [35]:
pyreadstat.write_sav(base_final, ruta_salida)

print(f" Archivo exportado correctamente: {ruta_salida}")

 Archivo exportado correctamente: base_final_unificada_normalizada.sav


In [36]:
import numpy as np
import pandas as pd

ruta_archivo = "base_final_unificada_normalizada.sav"

df, meta = pyreadstat.read_sav(ruta_archivo)

num_reales = ["EDADHOM", "EDADMUJ", "DIAOCU"]
for c in num_reales:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df.loc[df["EDADHOM"] >= 120, "EDADHOM"] = np.nan
df.loc[df["EDADMUJ"] >= 120, "EDADMUJ"] = np.nan
df.loc[(df["DIAOCU"] < 1) | (df["DIAOCU"] > 31), "DIAOCU"] = np.nan

df_num = df[num_reales].copy()

print("df shape:", df.shape)
print("df_num shape:", df_num.shape)



df shape: (71576, 23)
df_num shape: (71576, 3)


In [37]:
#Descripción General del conjunto de datos 
num_observaciones = df.shape[0]
num_variables = df.shape[1]

print("Cantidad de observaciones:", num_observaciones)
print("Cantidad de variables:", num_variables)

Cantidad de observaciones: 71576
Cantidad de variables: 23


In [38]:
# Significado y tipo de cada variable 

diccionario = pd.DataFrame({
    "Variable": df.columns,
    "Tipo_dato": [str(df[c].dtype) for c in df.columns],
    "Tipo_variable": ["Categórica"] * len(df.columns)
})

numericas = ["EDADHOM", "EDADMUJ", "DIAOCU"]
diccionario.loc[diccionario["Variable"].isin(numericas), "Tipo_variable"] = "Numérica"

diccionario


,Variable,Tipo_dato,Tipo_variable
0,DEPREG,object,Categórica
1,MUPREG,object,Categórica
2,MESREG,object,Categórica
3,AÑOREG,float64,Categórica
4,DIAOCU,float64,Numérica
5,MESOCU,object,Categórica
6,AÑOOCU,float64,Categórica
7,DEPOCU,object,Categórica
8,MUPOCU,object,Categórica
9,EDADHOM,float64,Numérica


In [39]:
#Exploración de variables numéricas - tendencia central
tendencia_central = pd.DataFrame({
    "media": df_num.mean(numeric_only=True),
    "mediana": df_num.median(numeric_only=True),
    "moda": df_num.mode(numeric_only=True).iloc[0]
})

tendencia_central


,media,mediana,moda
EDADHOM,36.024360,34.0,31.0
EDADMUJ,32.802849,31.0,27.0
DIAOCU,15.733584,16.0,22.0


In [40]:
#Distribución y medidas de dispersión
distribucion = pd.DataFrame({
    "min": df_num.min(numeric_only=True),
    "max": df_num.max(numeric_only=True),
    "rango": df_num.max(numeric_only=True) - df_num.min(numeric_only=True),
    "varianza": df_num.var(numeric_only=True),
    "desviacion_std": df_num.std(numeric_only=True),
    "coef_variacion": df_num.std(numeric_only=True) / df_num.mean(numeric_only=True)
})

orden = df_num.quantile([0.25, 0.5, 0.75]).T
orden.columns = ["Q1_25%", "Q2_50%_mediana", "Q3_75%"]

forma = pd.DataFrame({
    "skewness": df_num.skew(numeric_only=True),
    "kurtosis": df_num.kurtosis(numeric_only=True)
})

distribucion, orden, forma


(          min   max  rango    varianza  desviacion_std  coef_variacion
 EDADHOM  15.0  98.0   83.0  100.558787       10.027900        0.278364
 EDADMUJ  15.0  81.0   66.0   83.123230        9.117194        0.277939
 DIAOCU    1.0  31.0   30.0   75.399772        8.683304        0.551896,
          Q1_25%  Q2_50%_mediana  Q3_75%
 EDADHOM    29.0            34.0    41.0
 EDADMUJ    26.0            31.0    38.0
 DIAOCU      8.0            16.0    23.0,
          skewness  kurtosis
 EDADHOM  1.168343  1.660497
 EDADMUJ  1.045180  1.272305
 DIAOCU   0.007677 -1.189502)

In [41]:
def tabla_freq(col, top=10):
    t = df[col].value_counts(dropna=False).head(top).to_frame("frecuencia")
    t["porcentaje"] = (t["frecuencia"] / len(df) * 100).round(2)
    return t

freq_DEPREG = tabla_freq("DEPREG", top=10)
freq_DEPOCU = tabla_freq("DEPOCU", top=10)
freq_MESOCU = tabla_freq("MESOCU", top=12)
freq_ESCHOM = tabla_freq("ESCHOM", top=10)
freq_ESCMUJ = tabla_freq("ESCMUJ", top=10)

freq_DEPREG, freq_DEPOCU, freq_MESOCU, freq_ESCHOM, freq_ESCMUJ




(                frecuencia  porcentaje
 DEPREG                                
 Guatemala            27573       38.52
 Quetzaltenango        5771        8.06
 Escuintla             3187        4.45
 San Marcos            2766        3.86
 Jutiapa               2629        3.67
 Huehuetenango         2381        3.33
 Izabal                2255        3.15
 Retalhuleu            2116        2.96
 Santa Rosa            1981        2.77
 Suchitepequez         1926        2.69,
                 frecuencia  porcentaje
 DEPOCU                                
 Guatemala            26530       37.07
 Quetzaltenango        5437        7.60
 Escuintla             3481        4.86
 Jutiapa               3064        4.28
 San Marcos            3041        4.25
 Huehuetenango         2480        3.46
 Izabal                2313        3.23
 Suchitepequez         2108        2.95
 Retalhuleu            2072        2.89
 Santa Rosa            2051        2.87,
             frecuencia  porcentaje
 M

In [42]:
# 1) Correlación EDADHOM vs EDADMUJ
corr_edades = df_num[["EDADHOM","EDADMUJ"]].corr()

# 2) Diferencia de edades
df["DIF_EDAD"] = df["EDADHOM"] - df["EDADMUJ"]
res_dif = df["DIF_EDAD"].describe()

# 3) Divorcios por mes de ocurrencia (frecuencia)
divorcios_por_mes = df["MESOCU"].value_counts().sort_index().to_frame("frecuencia")
divorcios_por_mes["porcentaje"] = (divorcios_por_mes["frecuencia"] / len(df) * 100).round(2)

corr_edades, res_dif, divorcios_por_mes


(          EDADHOM   EDADMUJ
 EDADHOM  1.000000  0.792371
 EDADMUJ  0.792371  1.000000,
 count    34359.000000
 mean         3.162810
 std          6.062616
 min        -41.000000
 25%          0.000000
 50%          2.000000
 75%          6.000000
 max         75.000000
 Name: DIF_EDAD, dtype: float64,
             frecuencia  porcentaje
 MESOCU                            
 Abril             5720        7.99
 Agosto            6620        9.25
 Diciembre         3854        5.38
 Enero             5351        7.48
 Febrero           6378        8.91
 Julio             6389        8.93
 Junio             6119        8.55
 Marzo             6989        9.76
 Mayo              6269        8.76
 Noviembre         4883        6.82
 Octubre           7003        9.78
 Septiembre        6001        8.38)